In [2]:
import os
from lxml import etree
import lxml.html
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from user_agent import generate_user_agent

import requests
from tqdm import tqdm
import time
import json
import random

import pandas as pd

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
co = webdriver.ChromeOptions()
# co.add_argument('--headless')  # 无界面模式
co.add_argument('--disable-images')
# co.add_argument('--disable-javascript') #禁用js
# co.add_argument('--proxy-server=http://ip:port') #使用代理
co.add_argument('--user-agent={}'.format(generate_user_agent(navigator=['chrome'])))

# 自动安装 webdriver

自动安装webdriver参见：https://stackoverflow.com/questions/60296873/sessionnotcreatedexception-message-session-not-created-this-version-of-chrome/62127806

In [5]:
browser = webdriver.Chrome(executable_path=ChromeDriverManager().install(),options=co,service_log_path=os.path.devnull)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/taptap/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


# 抓取租房贴

需要登录，否则大概没几页就提示登录了

In [6]:
url = 'https://www.douban.com/group/shanghaizufang/discussion?start=0'

browser.get(url)

In [29]:
def get_topics(url):
    
    browser.get(url)

    html = browser.page_source

    s = etree.HTML(html)

    url_ls = s.xpath('//table[@class="olt"]//tr[not(contains(@class,"th"))]//td[@class="title"]/a/@href')

    title_ls = s.xpath('//table[@class="olt"]//tr[not(contains(@class,"th"))]//td[@class="title"]/a/@title')
    author_ls = s.xpath('//table[@class="olt"]//tr[not(contains(@class,"th"))]//td[not(@class)]/a/text()')
    author_url_ls = s.xpath('//table[@class="olt"]//tr[not(contains(@class,"th"))]//td[not(@class)]/a/@href')
    last_response = s.xpath('//table[@class="olt"]//tr[not(contains(@class,"th"))]//td[@class="time"]/text()')

    df = pd.DataFrame({'url':url_ls,'title':title_ls,'author':author_ls,'author_url':author_url_ls,'last_response':last_response})
    
    return df

In [28]:
df_ls = []

In [ ]:
for idx in tqdm(range(290,500)):
    start = idx*25
    url = f'https://www.douban.com/group/shanghaizufang/discussion?start={start}'
    df = get_topics(url)
    if len(df)>0:
        df_ls.append(df)
        time.sleep(random.choice(range(1,4)))
    else:
        print(idx)
        break

 21%|██▏       | 45/210 [02:00<07:46,  2.82s/it]

In [35]:
len(df_ls)

290

In [100]:
df_res = pd.concat(df_ls)

df_res.drop_duplicates(inplace=True)

df_res.shape

(7163, 3)

# 过滤一下

In [93]:
import re

In [102]:
flag_zhuanzu =  df_res['title'].apply(lambda x:'转' in x)

flag_subway = df_res['title'].apply(lambda x:re.search('虹桥路|吴中路|漕河泾开发区|姚虹路',x))


In [103]:
df_res[flag_zhuanzu&flag_subway]

,url,title,last_response
15,https://www.douban.com/group/topic/215084766/,【个人转租】9号线漕河泾开发区站虹兴苑小区 朝南大卧室只租女生,03-24 17:19
12,https://www.douban.com/group/topic/216853903/,个人转租，闵行徐汇湘府花园吴中路姚虹路宋园路,03-24 15:47
20,https://www.douban.com/group/topic/216932044/,个人转租万源新城 9号线合川路、漕河泾开发区近字节跳动,03-24 09:30
15,https://www.douban.com/group/topic/215950506/,【个人转租】3、4、10号线虹桥路，仅限女生！,03-24 07:41
10,https://www.douban.com/group/topic/213904892/,9号线 桂林路地铁站 15号线 吴中路地铁站 田林 桂林苑 钦州北路888弄 次卧【转租】,03-23 19:53
24,https://www.douban.com/group/topic/216255954/,转租合川路地铁站附近近字节，漕河泾开发区可短租4个月,03-23 19:49
16,https://www.douban.com/group/topic/216585832/,闵行区吴中路朗诗寓转租,03-23 17:52


In [104]:
browser.quit()